# Setup

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 71.6 MB/s 
     |████████████████████████████████| 77 kB 8.9 MB/s 
     |████████████████████████████████| 6.6 MB 70.5 MB/s 
     |████████████████████████████████| 895 kB 82.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 212 kB 22.7 MB/s 
     |████████████████████████████████| 136 kB 61.2 MB/s 
     |████████████████████████████████| 1.1 MB 63.3 MB/s 
     |████████████████████████████████| 127 kB 85.9 MB/s 
     |████████████████████████████████| 271 kB 79.7 MB/s 
     |████████████████████████████████| 144 kB 78.1 MB/s 
     |████████████████████████████████| 94 kB 4.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [2]:
import numpy as np
import sklearn
import gensim
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [66]:
# constants and global variables
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CIL/Dataset/{}'
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/CIL/Models/{}'
PROBABILITIES = '/content/drive/MyDrive/Colab Notebooks/CIL/Probabilities/{}'

# if set to true the trainig of the classifier models will be performed, otherwise the 
# models will be loaded from a file (if present)
is_train_enabled = True

# Read Preprocessed Data

In [98]:
def load_X_data(file_name):
  tweets = []
  with open(DATA_PATH.format(file_name), 'r', encoding='utf-8') as f:
      for line in f:
        tweets.append(line.rstrip())
  
  return np.array(tweets)

X_train = load_X_data("X_train_processed_bert.txt")
X_test = load_X_data("X_test_processed_bert.txt")

In [99]:
def load_y_data(file_name):
  labels = []
  with open(DATA_PATH.format(file_name), 'r', encoding='utf-8') as f:
      for line in f:
        labels.append(int(line.rstrip()))
  
  return np.array(labels)

y_train = load_y_data("y_train.txt")

In [100]:
for i in range(10):
  print(X_train[i])
  print(X_test[i])
  print(y_train[i])

<user> yes i didn't even realize he did ! i really wanted him to respond because i want to buy him candy or something
sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air
1
bradly james lowrey is my bestfriend & & & he mean alot to me
<user> shucks well i work all week so now i can't come cheer you on ! oh and put those batteries in your calculator ! ! !
1
mckleinusa ashburn 15144 s series leather laptop case ( brown clean , front flap-over design with a secure key l ... <url>
i cant stay away from bug thats my baby
0
<user> next time ima come in yo class nd wake you up myself ! i wanted you to come out , so i can get my hug but you was sleep
<user> no ma'am ! ! ! lol im perfectly fine and not contagious anymore lmao
0
trivial pursuit for juniors ( second edition the second edition for juniors of the legendary trivial pursuit games <url>
whenever i fall asleep watching the tv
0
new adds please pin : 28b9ead0
<user> he needs to get rid of that thing ! it scares 

# BERT

In [101]:
from sklearn.model_selection import train_test_split

# split the model into training test and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=33)

In [102]:
from datasets import Dataset

# go from list type to 'Dataset' type. This object is requiered to train the model
df = pd.DataFrame(X_train, columns =['Phrase'])
train = Dataset.from_pandas(df).add_column(name="Label", column=y_train)

df = pd.DataFrame(X_val, columns =['Phrase'])
val = Dataset.from_pandas(df).add_column(name="Label", column=y_val)

df = pd.DataFrame(X_test, columns =['Phrase'])
test = Dataset.from_pandas(df)

In [103]:
# perform tokenization of the dataset
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(sentence):
    return tokenizer(sentence['Phrase'], padding=True, truncation=True, max_length=30)

train = train.map(tokenize_function, batched=True)
test = test.map(tokenize_function, batched=True)
val = val.map(tokenize_function, batched=True)

for i in range(5):
  print(train[i])
  print(test[i])
  print(val[i])

  0%|          | 0/2500 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

{'Phrase': "<user> yes i didn't even realize he did ! i really wanted him to respond because i want to buy him candy or something", 'Label': 1, 'input_ids': [101, 133, 4795, 135, 4208, 178, 1238, 112, 189, 1256, 4663, 1119, 1225, 106, 178, 1541, 1458, 1140, 1106, 6297, 1272, 178, 1328, 1106, 4417, 1140, 13962, 1137, 1380, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'Phrase': 'sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air', 'input_ids': [101, 2343, 1202, 1186, 5250, 2343, 188, 2528, 26950, 113, 2865, 1114, 1103, 15139, 2343, 118, 1202, 1186, 17043, 2528, 21670, 8308, 1586, 102, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [104]:
# go from 'Dataset' type to tensorflow so that our dataset can be used for training in keras
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

tf_train_dataset = train.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["Label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=64,
)

tf_val_dataset = val.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["Label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=64,
)

tf_test_dataset = test.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=64,
)


In [105]:
from transformers import TFAutoModelForSequenceClassification

# Compute some variables needed to speed up training
train_steps_per_epoch = int(len(tf_train_dataset) * (100/100) / 64)
dev_steps_per_epoch = int(len(tf_val_dataset) * (100/100) / 64)


# download pre-trained model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [106]:
if is_train_enabled:
  # train model
  model.fit(tf_train_dataset,
            validation_data=tf_val_dataset,
            epochs=6,
            verbose=2,
            steps_per_epoch=train_steps_per_epoch,
            validation_steps=dev_steps_per_epoch,)
  
  # save model to file
  model.save_pretrained(MODEL_PATH.format("bert"))

Epoch 1/6
610/610 - 160s - loss: 0.3839 - sparse_categorical_accuracy: 0.8214 - 160s/epoch - 262ms/step
Epoch 2/6
610/610 - 144s - loss: 0.3403 - sparse_categorical_accuracy: 0.8471 - 144s/epoch - 236ms/step
Epoch 3/6
610/610 - 144s - loss: 0.3346 - sparse_categorical_accuracy: 0.8499 - 144s/epoch - 236ms/step
Epoch 4/6
610/610 - 144s - loss: 0.3243 - sparse_categorical_accuracy: 0.8553 - 144s/epoch - 236ms/step
Epoch 5/6
610/610 - 144s - loss: 0.3247 - sparse_categorical_accuracy: 0.8562 - 144s/epoch - 236ms/step
Epoch 6/6
610/610 - 144s - loss: 0.3133 - sparse_categorical_accuracy: 0.8625 - 144s/epoch - 236ms/step


In [107]:
# load trained model from file
model.load_weights(MODEL_PATH.format("bert/tf_model.h5"))

In [108]:
y_pred = model.predict(tf_test_dataset)[0]

# write prediction probabilites to a file. These probabilites will be used to create
# an ensemble of models
def write_output_probas(file_name, Y):
  f  = open(file_name, "w")
  f.write("Id,Prediction\n")
  id = 1
  for y in Y:
    f.write(str(id) + "," + str(list(y)) + "\n")
    id = id + 1
  f.close()

write_output_probas(PROBABILITIES.format("bert_predictions_probas.csv"), y_pred)


# write predictions to file
y_pred = np.argmax(np.array(y_pred), axis=-1)
def write_output(file_name, Y):
  f  = open(file_name, "w")
  f.write("Id,Prediction\n")
  id = 1
  for y in Y:
    if y == 0:
      y = -1
    f.write(str(id) + "," + str(y) + "\n")
    id = id + 1
  f.close()

write_output(DATA_PATH.format("bert_predictions.csv"), y_pred)